<a href="https://colab.research.google.com/github/kgautam2103/zoom_participant_list_public/blob/main/one_time_events_zoom_report_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
from google.colab import drive
drive.mount("mnt",force_remount=True)
%run '/content/mnt/MyDrive/Colab Notebooks/zoom_participant_list_utility_functions.ipynb'


Mounted at mnt


In [ ]:
import requests
import random
import json
import pytz
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import gspread
import re
from pandas import ExcelWriter
from google.colab import files
from gspread_dataframe import set_with_dataframe
from pandas import json_normalize
from urllib.parse import urlencode
!pip install python-Levenshtein
!pip install fuzzywuzzy
from fuzzywuzzy import process
#!pip install gspread-pandas
#from gspread_pandas import Spread, Client


In [ ]:
report_datetime= input("enter the report datetime(UTC) like 2021-07-11T02 : ")
report_date = report_datetime[:-3]
event_name = input("enter the event name like NA_NYE_2022 : ")

meetingID = '9181716151'
page_size ='300'
base_url = 'https://api.zoom.us/v2/'
past_meetings = base_url+'/past_meetings/'
poll1_id= 'l1gX0McLRc6mxffk37nnIw'
poll2_id = 'mlE7n7f3TQ-qgXoF--MKPQ'


#to get result for the last held meeting
report_meetings = base_url+'report/meetings/'
get_meeting_url = report_meetings+meetingID

get_meeting_url = past_meetings+meetingID+'/instances'
get_user_url = base_url+'users/'

#auth token for zoom account admin user
auth_token = ''

authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

meeting_id_response = requests.get(get_meeting_url,headers=auth_headers)


if meeting_id_response.status_code == 200:

  for x in meeting_id_response.json()['meetings']:
    if report_datetime in str(x['start_time']):
      meeting_uid = str(x['uuid'])
      print(x)

enter the report datetime(UTC) like 2021-07-11T02 : 2021-12-31T10
enter the event name like NA_NYE_2022 : APAC_NYE_2022
{'uuid': 'J5cvhCo2ROibo695X6sFdg==', 'start_time': '2021-12-31T10:35:56Z'}


In [ ]:
current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_xlsx = current_ts+'_participants_list_'+report_date+".xlsx"

wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1WhACIU6F7i9VwDxXoqzx6GEwutQI5HpIYl4v6H49-7Q/')

input_sheet = wb.worksheet('sahajyogis')
final_combined_sheet = wb.worksheet('combined_sheet')
rolling_3week_sheet = wb.worksheet('Rolling3Weekend')
weekend_archive_sheet = wb.worksheet('WeekendArchive')
Sideroom_sheet = wb.worksheet('regular_sideroom')
move2side_sheet = wb.worksheet('move2side')
updated_sideroom_sheet = wb.worksheet('updated_sideroom')
month_3_archive_frequency_sheet = wb.worksheet('3month_archive_frequency')

inputdata_df = read_worksheet(input_sheet,['id','name','email'])
rolling_3week_df = read_worksheet(rolling_3week_sheet,['event','event_date','id','name', 'user_email', 'join_time', 'leave_time', 'duration'])
weekend_archive_df = read_worksheet(weekend_archive_sheet,['event','event_date','id','name', 'user_email', 'join_time', 'leave_time', 'duration'])
sideroom_data_df = read_worksheet(Sideroom_sheet,['name'])


report_datetime = datetime.strptime(report_date, '%Y-%m-%d')
report_datetime_3week_before = report_datetime-timedelta(days=15)
report_datetime_3month_before = report_datetime-timedelta(days=90)

ref_id_list = list(filter(None, inputdata_df['id'].tolist()))
ref_name_list = list(filter(None, inputdata_df['name'].tolist()))
ref_name_list = [i.lower() for i in ref_name_list]
ref_email_list = list(filter(None, inputdata_df['email'].tolist()))
ref_email_list = [i.lower() for i in ref_email_list]


if meeting_uid != "":
  encoded_uid = double_encode_uuid(meeting_uid)
  participants_list = get_participants_list(encoded_uid)
  #poll_answer_df = get_poll_answer_df(encoded_uid)
  #poll1_df = get_poll1_df(poll_answer_df,poll1_id)
  #poll2_df = get_poll2_df(poll_answer_df,poll2_id)

if len(participants_list)>0:
  participants_df = get_participants_df(participants_list,event_name,ref_id_list,ref_name_list,ref_email_list)


final_combined_df = participants_df
final_combined_sheet.clear()
set_with_dataframe(final_combined_sheet, final_combined_df)

#if move2side_input == 'true':
rolling_3week_df_final = get_rolling_3week_df(rolling_3week_df,final_combined_df)
weekend_archive_df_final = get_weekend_archive_df(weekend_archive_df,final_combined_df)

weekend_archive_sheet.clear()
rolling_3week_sheet.clear()
#set_with_dataframe(rolling3week_output_sheet, rolling_3week_df_final)
set_with_dataframe(weekend_archive_sheet,weekend_archive_df_final)
set_with_dataframe(rolling_3week_sheet, rolling_3week_df_final)

weekend_archive_df_final_filtered_3m_frequency = get_weekend_archive_df_final_filtered_3m(weekend_archive_df_final)
set_with_dataframe(month_3_archive_frequency_sheet,weekend_archive_df_final_filtered_3m_frequency)

archive_group_df_max_date_details = get_archive_group_df_max_date_details(weekend_archive_df_final)
archive_name_list=archive_group_df_max_date_details['name'].tolist()
archive_name_list_new = [i for i in archive_name_list if len(i) > 1]
matched_name_list=[]
for index,row in sideroom_data_df.iterrows():
  highest_match = process.extractOne(row['name'],archive_name_list_new)
  matched_name_list.append([row['name'],highest_match[0],highest_match[1]])

updated_sideroom_details_df = get_updated_sideroom_details(matched_name_list,archive_group_df_max_date_details)

updated_sideroom_sheet.clear()
set_with_dataframe(updated_sideroom_sheet, updated_sideroom_details_df)
#set_with_dataframe(updated_regular_output_sheet, updated_sideroom_details_df)

final_max_count_filtered = pivot_rolling_3week_df(rolling_3week_df_final)

regular_matched_name_list=updated_sideroom_details_df['name'].tolist()
regular_matched_name_list_new = [i for i in regular_matched_name_list if len(i) > 1]
move2side_name_list=[]
for index,row in final_max_count_filtered.iterrows():
  highest_match_move2side = process.extractOne(row['name'],regular_matched_name_list_new)
  move2side_name_list.append([row['name'],highest_match_move2side[0],highest_match_move2side[1]])

move2side_final = get_move2side_final(move2side_name_list,final_max_count_filtered)

email_name_ref_df = get_email_name_ref_df(weekend_archive_df_final)

move2side_final_df_with_email = pd.merge(move2side_final,email_name_ref_df,how='left',on='name')
move2side_final_df_with_email.drop_duplicates(inplace=True)

move2side_sheet.clear()
set_with_dataframe(move2side_sheet, move2side_final_df_with_email)
#set_with_dataframe(move2side_output_sheet,move2side_final_df_with_email)

with ExcelWriter(output_xlsx) as writer:
  final_combined_df.to_excel(writer, sheet_name="final_combined")
  rolling_3week_df_final.to_excel(writer, sheet_name="3weekRollingWeekend")
  move2side_final_df_with_email.to_excel(writer, sheet_name="move2side")
  updated_sideroom_details_df.to_excel(writer, sheet_name="updated_regular_sideroom")
#poll1_df.to_excel(writer, sheet_name="poll1")
#poll2_df.to_excel(writer, sheet_name="poll2")


files.download(output_xlsx)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>